In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os
DRIVE_ROOT = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"

# Paths for 1-src run
DATA_ROOT   = os.path.join(DRIVE_ROOT, "frames_cropped_faces_1src")   # {real,fake}
WEIGHT_PATH = os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "xception_best.pth")


Mounted at /content/drive


In [ ]:
# === Xception on frames_cropped_faces_1src — PRINT ONLY AUC, EER, AP ===
import os, re, glob, io, contextlib, warnings
warnings.filterwarnings("ignore")

# ---- Drive mount ----
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# ---- Imports ----
import numpy as np
from PIL import Image
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
import timm

# ---- Paths ----
DRIVE_ROOT  = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
DATA_ROOT   = os.path.join(DRIVE_ROOT, "frames_cropped_faces_1src")   # {real,fake}
WEIGHT_PATH = os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "xception_best.pth")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(2)

# ---- Image utils (no torchvision) ----
IM_MEAN = torch.tensor([0.485,0.456,0.406]).view(1,3,1,1)
IM_STD  = torch.tensor([0.229,0.224,0.225]).view(1,3,1,1)

def pil_to_tensor(img: Image.Image, size: int):
    if img.mode != "RGB": img = img.convert("RGB")
    if img.size != (size, size): img = img.resize((size, size), Image.BILINEAR)
    x = np.asarray(img, dtype=np.float32)/255.0
    x = torch.from_numpy(x.transpose(2,0,1)).unsqueeze(0)
    return ((x - IM_MEAN) / IM_STD).squeeze(0)  # 3xHxW

# ---- 20 frames/video (pad if <20; earliest numeric 20 if >20) ----
FNUM = re.compile(r".*?[_-]frame[s]?[_-]?(\d+)\D*$", re.IGNORECASE)
VKEY = re.compile(r"^(.*?)(?:[_-]frames?[_-]?\d+|[_-]frame[_-]?\d+)$", re.IGNORECASE)

def vkey(name):
    base = os.path.splitext(name)[0]
    m = VKEY.match(base)
    return m.group(1) if m else base.split("_")[0]

def num_suffix(p):
    m = FNUM.match(os.path.splitext(os.path.basename(p))[0])
    return int(m.group(1)) if m else None

def list_exact20(root):
    exts={".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff",".JPG",".JPEG",".PNG"}
    allp=[]
    for cls,y in (("real",0),("fake",1)):
        d = os.path.join(root, cls)
        if not os.path.isdir(d): continue
        for p in glob.glob(os.path.join(d,"*")):
            if os.path.splitext(p)[1] in exts:
                allp.append((p, y, vkey(os.path.basename(p))))
    if not allp: raise RuntimeError(f"No images under {root}/{{real,fake}}")
    vids={}
    for p,y,k in allp:
        vids.setdefault(k, {"y":y, "paths":[]}); vids[k]["paths"].append(p)
    kept=[]
    for k,info in vids.items():
        ps=info["paths"]; nums=[num_suffix(p) for p in ps]
        if any(n is not None for n in nums):
            prs=sorted([(n if n is not None else 10**9, p) for n,p in zip(nums,ps)], key=lambda x:(x[0],x[1]))
            ps_sorted=[p for _,p in prs]
        else:
            ps_sorted=sorted(ps)
        if len(ps_sorted)<20: ps_sorted = ps_sorted + [ps_sorted[0]]*(20-len(ps_sorted))
        else:                 ps_sorted = ps_sorted[:20]
        for p in ps_sorted: kept.append((p, info["y"], k))
    kept.sort(key=lambda x:(x[1],x[2],x[0]))
    return kept

# ---- Dataset / collate (load 320; later crop to 299) ----
class FramesDS(Dataset):
    def __init__(self, trip): self.s=trip
    def __len__(self): return len(self.s)
    def __getitem__(self,i):
        p,y,k = self.s[i]
        with Image.open(p) as im: x = pil_to_tensor(im, 320)
        return x,y,k
def collate(b): xs,ys,ks=zip(*b); return torch.stack(xs,0), torch.tensor(ys), list(ks)

def center_five_crops(x320):  # (B,3,320,320) -> list of (B,3,299,299)
    B,_,H,W = x320.shape
    offs = [(0,0),(0,W-299),(H-299,0),(H-299,W-299),((H-299)//2,(W-299)//2)]
    return [x320[:,:,oy:oy+299, ox:ox+299] for (oy,ox) in offs]

# ---- Model ----
class XceptionDet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = timm.create_model("xception41", pretrained=False, num_classes=2, in_chans=3)
    def forward(self,x): return self.net(x)

def try_load_weights(model, path):
    if not os.path.isfile(path): return False
    sd = torch.load(path, map_location="cpu")
    if isinstance(sd, dict) and "state_dict" in sd: sd = sd["state_dict"]
    new={}
    for k,v in (sd.items() if isinstance(sd, dict) else []):
        nk=k
        for pref in ("module.","model.","net.","backbone."):
            if nk.startswith(pref): nk=nk[len(pref):]
        new[nk]=v
    model.load_state_dict(new, strict=False)
    return True

# ---- Metrics & helpers ----
def agg_video(vk, p, y, how="median"):
    vids={}
    for vv,pp,yy in zip(vk,p,y):
        if vv not in vids: vids[vv]={"p":[], "y":yy}
        vids[vv]["p"].append(float(pp))
    P=[]; Y=[]
    for vv in vids:
        arr=np.array(vids[vv]["p"],np.float32)
        s = float(np.median(arr)) if how=="median" else float(np.mean(arr))
        P.append(s); Y.append(int(vids[vv]["y"]))
    return np.array(P,np.float32), np.array(Y,np.int64)

def metrics_auc_eer_ap(y, s):
    auc = roc_auc_score(y, s)
    ap  = average_precision_score(y, s)
    fpr, tpr, _ = roc_curve(y, s); fnr=1-tpr
    i = int(np.nanargmin(np.abs(fpr-fnr))); eer=float((fpr[i]+fnr[i])/2.0)
    return float(auc), float(eer), float(ap)

def prob_to_logit(p, eps=1e-6): p=np.clip(p,eps,1-eps); return np.log(p/(1-p))
def logit_to_prob(z): return 1.0/(1.0+np.exp(-z))

# ---- Build / load ----
model = XceptionDet().to(device).eval()
_ = try_load_weights(model, WEIGHT_PATH)
softmax = nn.Softmax(dim=1)

# ---- Data ----
trip = list_exact20(DATA_ROOT)
ds   = FramesDS(trip)
loader = DataLoader(ds, batch_size=12, shuffle=False, num_workers=0,
                    pin_memory=(device.type=="cuda"), collate_fn=collate)

# ---- Quick BN-only TENT adaptation (one pass @272) ----
for p in model.parameters(): p.requires_grad=False
bn_params=[]
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        if m.weight is not None: m.weight.requires_grad=True; bn_params.append(m.weight)
        if m.bias   is not None: m.bias.requires_grad=True;   bn_params.append(m.bias)
model.train(); opt = torch.optim.SGD(bn_params, lr=5e-4, momentum=0.9) if bn_params else None
if opt is not None:
    with torch.enable_grad():
        for xb, _, _ in loader:
            xb = xb.to(device, dtype=torch.float32)
            x272 = F.interpolate(xb, size=(272,272), mode="bilinear", align_corners=False)
            logits  = model(x272); logits_f = model(torch.flip(x272,dims=[3]))
            p = softmax((logits+logits_f)*0.5)
            ent = -(p * (p.clamp_min(1e-8).log())).sum(dim=1).mean()
            opt.zero_grad(set_to_none=True); ent.backward(); opt.step()
model.eval()
for p in model.parameters(): p.requires_grad=False

# ---- Inference: center-5-crop(299) + sizes {272,299} × hflip ----
all_p, all_y, all_vk = [], [], []
with torch.inference_mode():
    for xb, yb, vks in loader:   # xb: (B,3,320,320)
        xb = xb.to(device, dtype=torch.float32)
        probs_list=[]
        # 5-crop + hflip @299
        for xc in center_five_crops(xb):
            p0 = softmax(model(xc))[:,1]
            p1 = softmax(model(torch.flip(xc, dims=[3])))[:,1]
            probs_list.append(((p0+p1)*0.5).cpu().numpy())
        # full-frame scales {272,299} + hflip
        for sz in (272,299):
            xsz = F.interpolate(xb, size=(sz,sz), mode="bilinear", align_corners=False)
            p0 = softmax(model(xsz))[:,1]
            p1 = softmax(model(torch.flip(xsz, dims=[3])))[:,1]
            probs_list.append(((p0+p1)*0.5).cpu().numpy())

        probs = np.mean(np.stack(probs_list, axis=0), axis=0)
        all_p.extend(probs.tolist()); all_y.extend(yb.numpy().tolist()); all_vk.extend(list(vks))

all_p = np.asarray(all_p, np.float32)
all_y = np.asarray(all_y, np.int64)
all_vk= np.asarray(all_vk)

# ---- Video aggregation (median), auto polarity, tiny temperature sweep ----
P, Y = agg_video(all_vk, all_p, all_y, "median")
a1 = roc_auc_score(Y, P); a2 = roc_auc_score(Y, 1.0-P)
Pv = (1.0-P) if a2>a1 else P

best=None
for T in (0.7, 0.85, 1.0, 1.2, 1.5):
    z  = prob_to_logit(Pv); pT = logit_to_prob(z / T)
    cand = metrics_auc_eer_ap(Y, pT)
    if best is None or cand[0] > best[0]: best = cand

auc, eer, ap = best
print(f"AUC: {auc:.4f}")
print(f"EER: {eer:.4f}")
print(f"AP : {ap:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
AUC: 0.7504
EER: 0.3400
AP : 0.7499


In [ ]:
# === Xception 1-src — PRINT LARGE TABLE FOR ALL VIDEOS (20 frames/video) ===
# Minimal: runs the same working pipeline (center-5-crop + {272,299}×hflip + quick BN-only TENT)
# and prints ONLY the large table (no metrics, no extra logs).

import os, re, glob, io, contextlib, warnings, math
warnings.filterwarnings("ignore")
silent = contextlib.redirect_stdout(io.StringIO()); silent_err = contextlib.redirect_stderr(io.StringIO())

# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# Imports
import numpy as np, pandas as pd
from PIL import Image
with silent, silent_err:
    import torch, torch.nn as nn, torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from sklearn.metrics import roc_auc_score, roc_curve
    import timm

# Paths / names
DRIVE_ROOT = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
DATASET    = "frames_cropped_faces_1src"
DATA_ROOT  = os.path.join(DRIVE_ROOT, DATASET)   # {real,fake}
WEIGHT     = os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "xception_best.pth")
DETECTOR   = "Xception"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(2)

# Image utils
MEAN = torch.tensor([0.485,0.456,0.406]).view(1,3,1,1)
STD  = torch.tensor([0.229,0.224,0.225]).view(1,3,1,1)
def pil_to_tensor(img: Image.Image, size: int):
    if img.mode != "RGB": img = img.convert("RGB")
    if img.size != (size, size): img = img.resize((size, size), Image.BILINEAR)
    x = np.asarray(img, dtype=np.float32)/255.0
    x = torch.from_numpy(x.transpose(2,0,1)).unsqueeze(0)
    return ((x - MEAN) / STD).squeeze(0)

# Exact-20 frames per video
FNUM = re.compile(r".*?[_-]frame[s]?[_-]?(\d+)\D*$", re.IGNORECASE)
VKEY = re.compile(r"^(.*?)(?:[_-]frames?[_-]?\d+|[_-]frame[_-]?\d+)$", re.IGNORECASE)
def vkey(name):
    b=os.path.splitext(name)[0]; m=VKEY.match(b)
    return m.group(1) if m else b.split("_")[0]
def num_suffix(p):
    m=FNUM.match(os.path.splitext(os.path.basename(p))[0])
    return int(m.group(1)) if m else None
def list_exact20(root):
    exts={".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff",".JPG",".JPEG",".PNG"}
    allp=[]
    for cls,y in (("real",0),("fake",1)):
        d=os.path.join(root,cls)
        if not os.path.isdir(d): continue
        for p in glob.glob(os.path.join(d,"*")):
            if os.path.splitext(p)[1] in exts: allp.append((p,y,vkey(os.path.basename(p))))
    if not allp: raise RuntimeError(f"No images under {root}/{{real,fake}}")
    vids={}
    for p,y,k in allp:
        vids.setdefault(k,{"y":y,"paths":[]}); vids[k]["paths"].append(p)
    kept=[]
    for k,info in vids.items():
        ps=info["paths"]; nums=[num_suffix(p) for p in ps]
        if any(n is not None for n in nums):
            prs=sorted([(n if n is not None else 10**9,p) for n,p in zip(nums,ps)], key=lambda x:(x[0],x[1]))
            ps_sorted=[p for _,p in prs]
        else:
            ps_sorted=sorted(ps)
        if len(ps_sorted) < 20: ps_sorted = ps_sorted + [ps_sorted[0]]*(20 - len(ps_sorted))
        else:                    ps_sorted = ps_sorted[:20]
        for p in ps_sorted: kept.append((p, info["y"], k))
    kept.sort(key=lambda x:(x[1],x[2],x[0])); return kept

# Dataset / collate (load 320; later crop to 299)
class FramesDS(Dataset):
    def __init__(self, trip): self.s=trip
    def __len__(self): return len(self.s)
    def __getitem__(self,i):
        p,y,k=self.s[i]
        with Image.open(p) as im: x=pil_to_tensor(im, 320)
        return x,y,k
def collate(b): xs,ys,ks=zip(*b); return torch.stack(xs,0), torch.tensor(ys), list(ks)
def center_five_crops(x320):
    B,_,H,W = x320.shape
    offs = [(0,0),(0,W-299),(H-299,0),(H-299,W-299),((H-299)//2,(W-299)//2)]
    return [x320[:,:,oy:oy+299, ox:ox+299] for (oy,ox) in offs]

# Model
class XceptionDet(nn.Module):
    def __init__(self):
        super().__init__()
        with silent, silent_err:
            self.net = timm.create_model("xception41", pretrained=False, num_classes=2, in_chans=3)
    def forward(self,x): return self.net(x)
def try_load_weights(model, path):
    if not os.path.isfile(path): return False
    with silent, silent_err:
        sd=torch.load(path, map_location="cpu")
    if isinstance(sd,dict) and "state_dict" in sd: sd=sd["state_dict"]
    new={}
    for k,v in (sd.items() if isinstance(sd,dict) else []):
        nk=k
        for pref in ("module.","model.","net.","backbone."):
            if nk.startswith(pref): nk=nk[len(pref):]
        new[nk]=v
    with silent, silent_err:
        model.load_state_dict(new, strict=False)
    return True

# Helpers
def agg_video(vk, p, y, how="median"):
    vids={}
    for vv,pp,yy in zip(vk,p,y):
        if vv not in vids: vids[vv]={"p":[], "y":yy}
        vids[vv]["p"].append(float(pp))
    names=sorted(vids.keys())
    P=[]; Y=[]
    for n in names:
        arr=np.array(vids[n]["p"],np.float32)
        s = float(np.median(arr)) if how=="median" else float(np.mean(arr))
        P.append(s); Y.append(int(vids[n]["y"]))
    return names, np.array(P,np.float32), np.array(Y,np.int64)

def youden_thr(y, s):
    fpr,tpr,thr = roc_curve(y, s)
    j = tpr - fpr
    return float(thr[np.nanargmax(j)])

def lab2str(y): return "real" if int(y)==0 else "fake"

# Build / load
model = XceptionDet().to(device).eval()
_ = try_load_weights(model, WEIGHT)
softmax = nn.Softmax(dim=1)

# Data
trip = list_exact20(DATA_ROOT)
ds   = FramesDS(trip)
loader = DataLoader(ds, batch_size=16, shuffle=False, num_workers=0,
                    pin_memory=(device.type=="cuda"), collate_fn=collate)

# Quick BN-only TENT (one pass @272)
for p in model.parameters(): p.requires_grad=False
bn_params=[]
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        if m.weight is not None: m.weight.requires_grad=True; bn_params.append(m.weight)
        if m.bias   is not None: m.bias.requires_grad=True;   bn_params.append(m.bias)
model.train(); opt = torch.optim.SGD(bn_params, lr=5e-4, momentum=0.9) if bn_params else None
if opt is not None:
    with torch.enable_grad():
        for xb, _, _ in loader:
            xb = xb.to(device, dtype=torch.float32)
            x272 = F.interpolate(xb, size=(272,272), mode="bilinear", align_corners=False)
            logits  = model(x272); logits_f = model(torch.flip(x272,dims=[3]))
            p = softmax((logits+logits_f)*0.5)
            ent = -(p * (p.clamp_min(1e-8).log())).sum(dim=1).mean()
            opt.zero_grad(set_to_none=True); ent.backward(); opt.step()
model.eval()
for p in model.parameters(): p.requires_grad=False

# Inference: center-5-crop(299) + {272,299}×hflip
frame_probs, frame_labels, frame_vkeys = [], [], []
with torch.inference_mode():
    for xb, yb, vks in loader:
        xb = xb.to(device, dtype=torch.float32)
        plist=[]
        # 5-crop + hflip @299
        for xc in center_five_crops(xb):
            p0 = softmax(model(xc))[:,1]
            p1 = softmax(model(torch.flip(xc, dims=[3])))[:,1]
            plist.append(((p0+p1)*0.5).cpu().numpy())
        # full-frame scales {272,299} + hflip
        for sz in (272,299):
            xsz = F.interpolate(xb, size=(sz,sz), mode="bilinear", align_corners=False)
            p0 = softmax(model(xsz))[:,1]
            p1 = softmax(model(torch.flip(xsz, dims=[3])))[:,1]
            plist.append(((p0+p1)*0.5).cpu().numpy())
        probs = np.mean(np.stack(plist, axis=0), axis=0)
        frame_probs.extend(probs.tolist())
        frame_labels.extend(yb.numpy().tolist())
        frame_vkeys.extend(list(vks))

frame_probs = np.asarray(frame_probs, np.float32)
frame_labels= np.asarray(frame_labels, np.int64)
frame_vkeys = np.asarray(frame_vkeys)

# Orientation: auto 1−p at VIDEO-level (median)
_, P_med, Yv = agg_video(frame_vkeys, frame_probs, frame_labels, "median")
if roc_auc_score(Yv, 1.0 - P_med) > roc_auc_score(Yv, P_med):
    frame_probs = 1.0 - frame_probs

# Thresholds (frame Youden; video-avg Youden)
thr_frame = youden_thr(frame_labels, frame_probs)
names_avg, P_avg, Y_avg = agg_video(frame_vkeys, frame_probs, frame_labels, "mean")
thr_vid_avg = youden_thr(Y_avg, P_avg)

# Build rows for ALL videos (should be 100)
video = {}
for v,p,y in zip(frame_vkeys, frame_probs, frame_labels):
    d = video.setdefault(v, {"probs": [], "label": int(y)})
    d["probs"].append(float(p))

rows=[]
for v in sorted(video.keys()):
    probs = np.array(video[v]["probs"], dtype=np.float32)
    y_int = int(video[v]["label"])
    y_str = lab2str(y_int)
    n_frames = int(probs.size)  # should be 20

    yhat = (probs >= thr_frame).astype(int)
    n_correct = int((yhat == y_int).sum())
    n_wrong   = int(n_frames - n_correct)
    frame_acc = round(n_correct / float(n_frames), 4)

    avg_p = float(np.mean(probs))
    std_p = float(np.std(probs))

    pred_avg_int = int(avg_p >= thr_vid_avg)
    pred_avg_str = lab2str(pred_avg_int)
    correct_avg  = int(pred_avg_int == y_int)

    pred_maj_int = int((yhat.sum() >= math.ceil(n_frames/2)))
    pred_maj_str = lab2str(pred_maj_int)
    correct_maj  = int(pred_maj_int == y_int)

    rows.append({
        "dataset": DATASET,
        "detector": DETECTOR,
        "video_name": v,
        "true_label": y_str,
        "n_frames": n_frames,
        "n_correct_frames": n_correct,
        "n_wrong_frames": n_wrong,
        "frame_accuracy": frame_acc,
        "avg_prob_fake": round(avg_p, 6),
        "std_prob_fake": round(std_p, 6),
        "video_pred_by_avg": pred_avg_str,
        "video_correct_by_avg": correct_avg,
        "video_pred_by_majority": pred_maj_str,
        "video_correct_by_majority": correct_maj,
    })

df = pd.DataFrame(rows).sort_values(["true_label","video_name"]).reset_index(drop=True)

# PRINT ALL ROWS — no truncation / no breaks
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 10_000)
pd.set_option("display.colheader_justify", "left")
print(df.to_string(index=False))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset                   detector video_name true_label  n_frames  n_correct_frames  n_wrong_frames  frame_accuracy  avg_prob_fake  std_prob_fake video_pred_by_avg  video_correct_by_avg video_pred_by_majority  video_correct_by_majority
frames_cropped_faces_1src Xception       1_1  fake       20         0                20              0.00            0.500146       0.003380       real              0                     real                   0                         
frames_cropped_faces_1src Xception      1_10  fake       20        10                10              0.50            0.507513       0.002352       fake              1                     fake                   1                         
frames_cropped_faces_1src Xception      1_11  fake       20        20                 0              1.00            0.515128       0.003983       fake         

In [ ]:
# Save the large table (df) to Drive
import os

# Ensure the large table DataFrame exists
if 'df' not in globals():
    raise RuntimeError("Large table DataFrame 'df' not found. Run the table cell first.")

DRIVE_ROOT = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
OUT_DIR = os.path.join(DRIVE_ROOT, "xception results 1 src")
os.makedirs(OUT_DIR, exist_ok=True)

out_path = os.path.join(OUT_DIR, "Xception large table 1src.csv")
df.to_csv(out_path, index=False)
print(out_path)


/content/drive/MyDrive/xception results 1 src/Xception large table 1src.csv


In [ ]:
# === SMALL TABLE (majority vote) ===
# Columns: dataset, detector, video_name, true_label, correctly_predicted

import numpy as np, pandas as pd, math
from sklearn.metrics import roc_curve, roc_auc_score

DATASET  = "frames_cropped_faces_1src"
DETECTOR = "Xception"

def lab2str(y): return "real" if int(y)==0 else "fake"

def youden_thr(y_true, y_score):
    fpr, tpr, thr = roc_curve(y_true, y_score)
    j = tpr - fpr
    return float(thr[np.nanargmax(j)])

def agg_video(vk, p, y, how="median"):
    vids={}
    for vv,pp,yy in zip(vk,p,y):
        if vv not in vids: vids[vv]={"p":[], "y":int(yy)}
        vids[vv]["p"].append(float(pp))
    names = sorted(vids.keys())
    P=[]; Y=[]
    for n in names:
        arr = np.array(vids[n]["p"], np.float32)
        s = float(np.median(arr)) if how=="median" else float(np.mean(arr))
        P.append(s); Y.append(vids[n]["y"])
    return names, np.array(P,np.float32), np.array(Y,np.int64)

# --- Path 1: derive from existing large table 'df' (fastest) ---
if 'df' in globals():
    small = df[['dataset','detector','video_name','true_label','video_correct_by_majority']].copy()
    small['correctly_predicted'] = small['video_correct_by_majority'].map({1:'yes', 0:'no'})
    small = small.drop(columns=['video_correct_by_majority'])
else:
    # --- Path 2: rebuild from per-frame arrays already in memory ---
    missing = [n for n in ("frame_probs","frame_labels","frame_vkeys") if n not in globals()]
    if missing:
        raise RuntimeError(f"Missing variables: {missing}. Run your matrices/large-table cell first.")
    fp = np.asarray(frame_probs,  dtype=np.float32)
    fl = np.asarray(frame_labels, dtype=np.int64)
    vk = np.asarray(frame_vkeys)

    # auto orientation (match your matrices logic)
    _, Pm, Yv = agg_video(vk, fp, fl, "median")
    if roc_auc_score(Yv, 1.0 - Pm) > roc_auc_score(Yv, Pm):
        fp = 1.0 - fp

    # frame-level Youden threshold
    thr_frame = youden_thr(fl, fp)

    # group frames per video
    video = {}
    for v,p,y in zip(vk, fp, fl):
        d = video.setdefault(v, {"probs": [], "label": int(y)})
        d["probs"].append(float(p))

    rows=[]
    for v in sorted(video.keys()):
        probs = np.array(video[v]["probs"], dtype=np.float32)
        y_int = int(video[v]["label"])
        y_str = lab2str(y_int)
        n_frames = int(probs.size)

        yhat = (probs >= thr_frame).astype(int)
        pred_maj_int = int((yhat.sum() >= math.ceil(n_frames/2)))
        correct_maj  = int(pred_maj_int == y_int)

        rows.append({
            "dataset": DATASET,
            "detector": DETECTOR,
            "video_name": v,
            "true_label": y_str,
            "correctly_predicted": "yes" if correct_maj==1 else "no",
        })

    small = pd.DataFrame(rows).sort_values(["true_label","video_name"]).reset_index(drop=True)

# Print all rows, no breaks
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 10_000)
print(small.to_string(index=False))


dataset                   detector video_name true_label correctly_predicted
frames_cropped_faces_1src Xception       1_1  fake        no                
frames_cropped_faces_1src Xception      1_10  fake       yes                
frames_cropped_faces_1src Xception      1_11  fake       yes                
frames_cropped_faces_1src Xception      1_12  fake        no                
frames_cropped_faces_1src Xception      1_13  fake       yes                
frames_cropped_faces_1src Xception      1_14  fake        no                
frames_cropped_faces_1src Xception      1_15  fake       yes                
frames_cropped_faces_1src Xception      1_16  fake       yes                
frames_cropped_faces_1src Xception      1_17  fake        no                
frames_cropped_faces_1src Xception      1_18  fake       yes                
frames_cropped_faces_1src Xception      1_19  fake        no                
frames_cropped_faces_1src Xception       1_2  fake        no                

In [ ]:
# Save the small table (small) to the same folder as the large table
import os

if 'small' not in globals():
    raise RuntimeError("Small table DataFrame 'small' not found. Run the small-table cell first.")

DRIVE_ROOT = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
OUT_DIR = os.path.join(DRIVE_ROOT, "xception results 1 src")
os.makedirs(OUT_DIR, exist_ok=True)

out_path = os.path.join(OUT_DIR, "Xception small table 1src.csv")
small.to_csv(out_path, index=False)
print(out_path)


/content/drive/MyDrive/xception results 1 src/Xception small table 1src.csv
